In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import meteva.base as meb
import meteva.method as mem
import meteva.product as mpd
import numpy as np
import datetime
import copy
import matplotlib.pyplot as plt
import time
import pandas as pd
import math
import matplotlib as mpl
import os

In [2]:
ip,port = meb.read_gds_ip_port(r"H:\test_data\input\meb\ip_port.txt")
now = datetime.datetime.now()
now = datetime.datetime(now.year,now.month,now.day,now.hour,0)
today = datetime.datetime(now.year,now.month,now.day,0,0)
station_alt = meb.read_station(meb.station_国家站)
station = station_alt.copy()
station["data0"] = meb.IV
id_dict = {"北京":54511,"哈尔滨":50953, "武汉":57494,"广州":59287}
id_list = []
for key in id_dict.keys():
    id_list.append(id_dict[key])
station_city = meb.in_id_list(station,id_list)

In [7]:
scmoc_all = pd.read_hdf(hdf_list[1],"df")

In [12]:
grapes_all = pd.read_hdf(r"O:\data\hdf\NWFD_SCMOC\TMP\2M_ABOVE_GROUND\20042408.h5","df")

In [13]:
meb.sele_by_para(grapes_all,id = 54511)

,level,time,dtime,id,lon,lat,scmoc
779,0,2020-04-24 08:00:00,0,54511,116.47,39.8,30.392045
779,0,2020-04-24 08:00:00,1,54511,116.47,39.8,30.392045
779,0,2020-04-24 08:00:00,2,54511,116.47,39.8,30.392045
779,0,2020-04-24 08:00:00,3,54511,116.47,39.8,20.576020
779,0,2020-04-24 08:00:00,4,54511,116.47,39.8,20.576020
779,0,2020-04-24 08:00:00,5,54511,116.47,39.8,20.576020
779,0,2020-04-24 08:00:00,6,54511,116.47,39.8,27.694002
779,0,2020-04-24 08:00:00,7,54511,116.47,39.8,27.694002
779,0,2020-04-24 08:00:00,8,54511,116.47,39.8,27.694002
779,0,2020-04-24 08:00:00,9,54511,116.47,39.8,28.358014


In [22]:

ip,port = meb.read_gds_ip_port(r"H:\test_data\input\meb\ip_port.txt")
now = datetime.datetime.now()
now = datetime.datetime(now.year,now.month,now.day,now.hour,0)
today = datetime.datetime(now.year,now.month,now.day,0,0)
station_alt = meb.read_station(meb.station_国家站)
station = station_alt.copy()
station["data0"] = meb.IV
id_dict = {"北京":54511,"哈尔滨":50953, "武汉":57494,"广州":59287}
id_list = []
for key in id_dict.keys():
    id_list.append(id_dict[key])
station_city = meb.in_id_list(station,id_list)
veri_day_count = 7

#读取grapes数据
dir_hdf = r"O:\data\hdf\GRAPES_GFS\TMP\2M_ABOVE_GROUND\YYMMDDHH.h5"
dir_nc = r"O:\data\grid\GRAPES_GFS\TMP\2M_ABOVE_GROUND\YYYYMMDD\YYMMDDHH.TTT.nc"
dir_gds = r"GRAPES_GFS\TMP\2M_ABOVE_GROUND\YYMMDDHH.TTT"
data_name = "grapes"
sta_list = []
dir_gds0,file = os.path.split(dir_gds)
gds_file_list = meb.tool.path_tools.get_gds_file_list_in_one_dir(ip,port,dir_gds0)
hour_list,dhour_list = mpd.get_hour_dhour_list(gds_file_list)
print(dhour_list)
print(int(dhour_list[-1]/24))
max_dd = veri_day_count + int(dhour_list[-1]/24)
if max_dd > veri_day_count * 2:
    max_dd = veri_day_count*2
print(max_dd)
for dd in range(max_dd):
    day2 = today - datetime.timedelta(days = dd)
    print(day2)
    for hour in hour_list:
        sta_1f = None
        time2 =  datetime.datetime(day2.year,day2.month,day2.day,hour,0)
        path_hdf = meb.tool.path_tools.get_path(dir_hdf,time2)
        print(path_hdf)
        if os.path.exists(path_hdf):
            sta_1f = pd.read_hdf(path_hdf,"df")
        else:
            sta_list_1f  = []
            all_exist = True
            exist_dh = []
            for dh in dhour_list:
                path_nc = meb.tool.path_tools.get_path(dir_nc,time2,dh)
                grd = None
                if os.path.exists(path_nc):
                    grd = meb.read_griddata_from_nc(path_nc)
                else:
                    path_gds = meb.tool.path_tools.get_path(dir_gds,time2,dh)
                    if path_gds in gds_file_list:
                        #判断是否在gds服务器中
                        grd = meb.read_griddata_from_gds(ip,port,path_gds)
                if grd is not None:
                    sta = meb.interp_gs_linear(grd,station)
                    meb.set_stadata_coords(sta,time = time2,dtime = dh,level = 0)
                    meb.set_stadata_names(sta,[data_name])
                    sta_list_1f.append(sta)
                    exist_dh.append(dh)
                else:
                    all_exist = False
            print(exist_dh)
            if(len(sta_list_1f)>0):
                sta_1f = pd.concat(sta_list_1f,axis = 0)
                meb.creat_path(path_hdf)
                if(all_exist):
                    sta_1f.to_hdf(path_hdf,"df")
        if sta_1f is not None:
            sta_list.append(sta_1f)
grapes_all = pd.concat(sta_list,axis = 0)

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 126, 132, 138, 144, 150, 156, 162, 168, 174, 180, 186, 192, 198, 204, 210, 216, 222, 228, 234, 240]
10
14
2020-04-24 00:00:00
O:\data\hdf\GRAPES_GFS\TMP\2M_ABOVE_GROUND\20042402.h5
[0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120]
O:\data\hdf\GRAPES_GFS\TMP\2M_ABOVE_GROUND\20042408.h5
[]
O:\data\hdf\GRAPES_GFS\TMP\2M_ABOVE_GROUND\20042414.h5
[]
O:\data\hdf\GRAPES_GFS\TMP\2M_ABOVE_GROUND\20042420.h5
[]
2020-04-23 00:00:00
O:\data\hdf\GRAPES_GFS\TMP\2M_ABOVE_GROUND\20042302.h5
[0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120]
O:\data\hdf\GRAPES_GFS\TMP\2M_ABOVE_GROUND\20042308.h5
O:\data\hdf

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 126, 132, 138, 144, 150, 156, 162, 168, 174, 180, 186, 192, 198, 204, 210, 216, 222, 228, 234, 240]
O:\data\hdf\GRAPES_GFS\TMP\2M_ABOVE_GROUND\20041614.h5
[0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120]
O:\data\hdf\GRAPES_GFS\TMP\2M_ABOVE_GROUND\20041620.h5
[0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 126, 132, 138, 144, 150, 156, 162, 168, 174, 180, 186, 192, 198, 204, 210, 216, 222, 228, 234, 240]
2020-04-15 00:00:00
O:\data\hdf\GRAPES_GFS\TMP\2M_ABOVE_GROUND\20041502.h5
[0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84

In [23]:
ip,port = meb.read_gds_ip_port(r"H:\test_data\input\meb\ip_port.txt")
now = datetime.datetime.now()
now = datetime.datetime(now.year,now.month,now.day,now.hour,0)
today = datetime.datetime(now.year,now.month,now.day,0,0)
station_alt = meb.read_station(meb.station_国家站)
station = station_alt.copy()
station["data0"] = meb.IV
id_dict = {"北京":54511,"哈尔滨":50953, "武汉":57494,"广州":59287}
id_list = []
for key in id_dict.keys():
    id_list.append(id_dict[key])
station_city = meb.in_id_list(station,id_list)
veri_day_count = 7

#读取grapes数据
dir_hdf = r"O:\data\hdf\NWFD_SCMOC\TMP\2M_ABOVE_GROUND\YYMMDDHH.h5"
dir_nc = r"O:\data\grid\NWFD_SCMOC\TMP\2M_ABOVE_GROUND\YYYYMMDD\YYMMDDHH.TTT.nc"
dir_gds = r"NWFD_SCMOC\TMP\2M_ABOVE_GROUND\YYMMDDHH.TTT"
data_name = "scmoc"
sta_list = []
dir_gds0,file = os.path.split(dir_gds)
gds_file_list = meb.tool.path_tools.get_gds_file_list_in_one_dir(ip,port,dir_gds0)
hour_list,dhour_list = mpd.get_hour_dhour_list(gds_file_list)
print(dhour_list)
print(int(dhour_list[-1]/24))
max_dd = veri_day_count + int(dhour_list[-1]/24)
if max_dd > veri_day_count * 2:
    max_dd = veri_day_count*2
print(max_dd)
for dd in range(max_dd):
    day2 = today - datetime.timedelta(days = dd)
    print(day2)
    for hour in hour_list:
        sta_1f = None
        time2 =  datetime.datetime(day2.year,day2.month,day2.day,hour,0)
        path_hdf = meb.tool.path_tools.get_path(dir_hdf,time2)
        print(path_hdf)
        if os.path.exists(path_hdf):
            sta_1f = pd.read_hdf(path_hdf,"df")
        else:
            sta_list_1f  = []
            all_exist = True
            exist_dh = []
            for dh in dhour_list:
                path_nc = meb.tool.path_tools.get_path(dir_nc,time2,dh)
                grd = None
                if os.path.exists(path_nc):
                    grd = meb.read_griddata_from_nc(path_nc)
                else:
                    path_gds = meb.tool.path_tools.get_path(dir_gds,time2,dh)
                    if path_gds in gds_file_list:
                        #判断是否在gds服务器中
                        grd = meb.read_griddata_from_gds(ip,port,path_gds)
                if grd is not None:
                    sta = meb.interp_gs_linear(grd,station)
                    meb.set_stadata_coords(sta,time = time2,dtime = dh,level = 0)
                    meb.set_stadata_names(sta,[data_name])
                    sta_list_1f.append(sta)
                    exist_dh.append(dh)
                else:
                    all_exist = False
            print(exist_dh)
            if(len(sta_list_1f)>0):
                sta_1f = pd.concat(sta_list_1f,axis = 0)
                meb.creat_path(path_hdf)
                if(all_exist):
                    sta_1f.to_hdf(path_hdf,"df")
        if sta_1f is not None:
            sta_list.append(sta_1f)
grapes_all = pd.concat(sta_list,axis = 0)

[3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147, 150, 153, 156, 159, 162, 165, 168, 171, 174, 177, 180, 183, 186, 189, 192, 195, 198, 201, 204, 207, 210, 213, 216, 219, 222, 225, 228, 231, 234, 237, 240]
10
14
2020-04-24 00:00:00
O:\data\hdf\NWFD_SCMOC\TMP\2M_ABOVE_GROUND\20042408.h5
[3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147, 150, 153, 156, 159, 162, 165, 168, 171, 174, 177, 180, 183, 186, 189, 192, 195, 198, 201, 204, 207, 210, 213, 216, 219, 222, 225, 228, 231, 234, 237, 240]
O:\data\hdf\NWFD_SCMOC\TMP\2M_ABOVE_GROUND\20042420.h5
[]
2020-04-23 00:00:00
O:\data\hdf\NWFD_SCMOC\TMP\2M_ABOVE_GROUND\20042308.h5
[3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45

[3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147, 150, 153, 156, 159, 162, 165, 168, 171, 174, 177, 180, 183, 186, 189, 192, 195, 198, 201, 204, 207, 210, 213, 216, 219, 222, 225, 228, 231, 234, 237, 240]
O:\data\hdf\NWFD_SCMOC\TMP\2M_ABOVE_GROUND\20041420.h5
[3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147, 150, 153, 156, 159, 162, 165, 168, 171, 174, 177, 180, 183, 186, 189, 192, 195, 198, 201, 204, 207, 210, 213, 216, 219, 222, 225, 228, 231, 234, 237, 240]
2020-04-13 00:00:00
O:\data\hdf\NWFD_SCMOC\TMP\2M_ABOVE_GROUND\20041308.h5
[3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 